# Abstract

In this tutorial, we will implement some algorithms that finding the similarity documents. The document dataset was crawled from Viblo website: $\textbf{viblo.asia}$

## Algorithms
1. TF-IDF + SVD
2. DOC2VEC

# Preprocessing Data

In [1]:
import gensim
import os
import collections
import smart_open
import random
import pickle
from pyvi import ViTokenizer, ViPosTagger
from tqdm import tqdm
import numpy as np

We will load data in data folder. The data in "contents_markdown_viblo.pkl" is a list of document.

In [2]:
with open('data/contents_markdown_viblo.pkl', 'rb') as f:
    content = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/contents_markdown_viblo.pkl'

In [6]:
print(type(content))
print(len(content))

<class 'list'>
14157


In [40]:
print(content[0])

### Trong phần này tôi sẽ giúp các bạn mới học `React Native` hiểu qua phần cài đặt project đã tồn tại và cùng nhau tìm hiểu khái niệm cơ bản nhất của React Native.
# I. Một số chia sẻ cá nhân

## 1. Yêu cầu thứ 1
Yêu cầu để thực hiện theo bài viết này, bạn hãy cài đặt môi trường theo bài viết dưới đây

### [Học React Native từ cơ bản đến nâng cao - Phần 1 Hướng dẫn cài đặt và chạy "Hello world"](https://viblo.asia/p/hoc-react-native-tu-co-ban-den-nang-cao-phan-1-huong-dan-cai-dat-va-chay-hello-world-RQqKLYW0Z7z)

## 2. Yêu cầu thứ 2
Khi muốn code React Native bạn sẽ phải chọn cho mình 1 Editors phù hợp cho mình

Bạn có thể tham khảo các Editors theo link dưới đây

### [Top 10 Editors For React Native Mobile App Development](https://www.icicletech.com/blog/top-10-editors-for-react-native)

Còn lựa chọn của tôi là [**`Visual Studio Code`**](https://code.visualstudio.com/) vì đơn giản là dùng khá nhẹ và tiện dụng

![](https://i.imgur.com/ZfAI0VK.png)

Nếu bạn cũng chọn **`Visual Studio C

We will remove documents which it's text length is less than 1000. Because many documents do not have any meaning.

In [7]:
documents = []
for c in content:
    if len(c) > 1000:
        documents.append(c)

In [8]:
print(len(documents))

13887


# Implementation

## TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#### Get the corpus used in TfIdf model

In [10]:
def get_corpus(documents):
    corpus = []
    for i in tqdm(range(len(documents))):
        doc = documents[i]
        # preprocessing text, remove all letters which is not character
        doc = gensim.utils.simple_preprocess(doc)
        # because these documents is mostly written in Vietnamese
        # we need to use an library which supporting Vietnamese
        doc = ' '.join(doc)
        words = ViTokenizer.tokenize(doc)
        sentence = ' '.join(words)

        corpus.append(sentence)
    return corpus

In [11]:
corpus = get_corpus(documents)

100%|██████████| 13887/13887 [05:11<00:00, 44.63it/s]


#### Custom the tokenizer to use in TfidfVectorizer()

In [15]:
def my_tokenizer(doc):
#     doc = ViTokenizer.tokenize(doc)
    doc = doc.split(" ")
    return doc

#### Create an Vectorize object with max features (number of words in dictionary) to be 20000

In [16]:
vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, max_features=20000)

#### Train model

In [17]:
res_corpus = vectorizer.fit_transform(corpus)

In [19]:
print(len(vectorizer.get_feature_names()))


20000


## SVD

#### Reduce document vector's dimensions

In [20]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300, random_state=42)

In [21]:
truncated_corpus = svd.fit_transform(res_corpus)

In [22]:
print(truncated_corpus.shape)

(13887, 300)


#### Save model

In [24]:
filename = 'tfidf_svd_20k_to_128.pkl'
pickle.dump(svd, open(filename, 'wb'))
print("SAVED SUCESS")

SAVED SUCESS


#### Calculate similarity values between documents

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
similarity_matrix = cosine_similarity(truncated_corpus, truncated_corpus)

#### Test for first document, get 10 most similar document

In [34]:
np.argsort(similarity_matrix[0])[-10:]

array([ 3515,   752,  9364, 12871, 11940,  1489,  2265, 13167, 11466,
           0])

In [39]:
print(corpus[11466])

react native là gì react native là một frameworkework cho phép bạn xây_dựng một ứng_dụng trên native platforms sử_dụng kinh_nghiệm của lập_trình_viên bằng javascript và react http_facebook github io react trọng_tâm của react native là hiệu_quả của nhà phát_triển trên tất_cả các nền_tảng mà bạn quan_tâm học một lần viết trên bất_cứ nền_tảng nào facebook sử_dụng react native trên nhiều ứng_dụng và tiếp_tục đầu_tư vào react native native components các thành_phần native với react native bạn có_thể sử_dụng các thành_phần cơ_bản mặc_định của nền_tảng như uitabbar trên ios và drawer trên android điều này cho phép ứng_dụng có sự nhất_quán với phần còn lại của nền_tảng và giữ cho chất_lượng sản_phẩm cao các thành_phần này có_thể dễ_dàng tích_hợp vào ứng_dụng của bạn sử_dụng react component counterparts chẳng_hạn như tabbarios and javascript ios import react component tabbarios navigatorios from react native class app extends component render return tabbarios tabbarios item title react native s

## Doc2Vec

#### Create Corpus which is trained in Doc2Vec Model

In [9]:
def get_training_corpus(documents):
    corpus = []
    for i in tqdm(range(len(documents))):
        doc = documents[i]
        # preprocessing text, remove all letters which is not character
        doc = gensim.utils.simple_preprocess(doc) 
        # because these documents is mostly written in Vietnamese
        # we need to use an library which supporting Vietnamese
        doc = ' '.join(doc)
        words = ViTokenizer.tokenize(doc)
        # convert to format used in Doc2Vec function below
        tagged_document = gensim.models.doc2vec.TaggedDocument(words.split(" "), [i])
        corpus.append(tagged_document)
    return corpus

In [ ]:
train_corpus = get_training_corpus(documents)

#### Build Doc2Vec Model

In [ ]:
# build a Doc2Vec model with vector size 300
# remove all the words which occur less than 2 times
# training in 40 epochs
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)
model.build_vocab(train_corpus)

In [ ]:
# train model and get the training time
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

#### Test for first document

In [ ]:
# get the vector for first document
vector = model.infer_vector(train_corpus[0].words)

In [ ]:
# get first 100 similar documents to first document
sims = model.docvecs.most_similar([vector], topn=100)

In [ ]:
for i in range(10):
    print(i, " : ", sims[i][0])